In [235]:
#importing libraries

import pandas as pd
import numpy as np

energy_data = pd.read_csv('C:/Users/HP/Downloads/energydata_complete.csv')
energy_df = energy_data.drop(columns = ['date'])



The percent of the total variation of the dependent variable Y explained by the set of independent variables X is measured by 'coefficient of determination'

A residual is the difference between an observed Y(response variable) and a predicted ^Y

The slope of a straight line graph is represented by Y = mx + c ; where m is the slope, similar to  Y = a + BX

Heteroskedasticity explains the variance in error terms

Linear regression is used to measure continuos variable while logistic variable measures classification problem

R Squared, Adjusted R Squared, F Statistics , RMSE / MSE / MAE can all be used to evaluate regression models 


A best fit line relating X and Y of a R-Squared value of 0.75 means 75% of the variance in Y is explained by X

The Lasso can be interpreted as least-squares linear regression where: Weights are regularized with the L1 norm

In [236]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_df), columns=energy_df.columns)

X = normalised_df['T2'].values.reshape(-1,1)
Y = normalised_df['T6'].values.reshape(-1,1)


x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2_score = r2_score(y_test,y_pred)
print('r2_score is : {}'.format(r2_score))


r2_score is : 0.6427789637197382


In [237]:
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_df), columns=energy_df.columns)
X = normalised_df.drop(columns=['Appliances','lights'])
Y = normalised_df['Appliances']

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('The mean absolute error is: {}'.format(round(mae, 2)))

The mean absolute error is: 0.05


In [238]:
rss = np.sum(np.square(y_test - y_pred))
print('The residual sum of squares is : {}'.format(round(rss, 2)))

The residual sum of squares is : 45.35


In [239]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test,y_pred,squared =False)
print('Root mean squared error is :{}'.format(round(rmse,3)))


Root mean squared error is :0.088


In [240]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)
print('The coefficient of determination is : {}'.format(round(r2_score, 2)))

The coefficient of determination is : 0.15


In [241]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
lasso_reg.fit(x_train, y_train)
y_pred = lasso_reg.predict(x_test)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print('Root mean squared error is :{}'.format(round(rmse,3)))
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['X', col_name]
    weights_df[col_name].round(3)
    return weights_df
linear_model_weights = get_weights_df(model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='X')
final_weights = pd.merge(final_weights, lasso_weights_df, on='X')
display(final_weights)


Root mean squared error is :0.094


,X,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


In [242]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
y_pred = ridge_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root mean squared error is :{}'.format(round(rmse,3)), ', there is therefore change in root mean squared error')




Root mean squared error is :0.088 , there is therefore change in root mean squared error


In [243]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)


#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['X', col_name]
    weights_df[col_name].round(3)
    return weights_df
linear_model_weights = get_weights_df(model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='X')
final_weights = pd.merge(final_weights, lasso_weights_df, on='X')
display(final_weights)



,X,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


In [244]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
lasso_reg.fit(x_train, y_train)
y_pred = lasso_reg.predict(x_test)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print('Root mean squared error is :{}'.format(round(rmse,3)))



Root mean squared error is :0.094
